# Webapp: set up data stream, visualize, and equitize

- https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
- https://dash.plotly.com/live-updates
- https://pbpython.com/interactive-dashboards.html#id6
- https://mybinder.org/ + https://github.com/echow1/trading_music
- (maybe) https://www.freecodecamp.org/news/how-to-create-auto-updating-data-visualizations-in-python-with-matplotlib-and-aws/
- https://kapernikov.com/ipywidgets-with-matplotlib/

In [ ]:
from __future__ import division
from more_itertools import peekable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scp
import magenta
import os, time, re
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

PLOT_WIDTH = 1200
PLOT_HEIGHT = 800

def hheader(x):
    print("#########################################")
    print("### {}".format(x))
    print("#########################################")

# Magenta dependencies:
# https://github.com/magenta/magenta

# Magenta uses pretty_midi to deal with midi files
import pretty_midi

### Setup: read in the music stream

In [ ]:
""" Set up music stream """

### for reading in chunks
from collections import deque

def csvStream(csvfile):
    csv_stream = pd.read_csv(csvfile, index_col=0, iterator=True)
    return(csv_stream)

def nextChunk(csvStream, chunksize=5):
    return(csvStream.get_chunk(chunksize))

chunksQueue = []
def nextChunkWithOverlap(musStream, cq=chunksQueue, chunksize=10, maxNumChunks=5):
    nextChunk = (musStream.get_chunk(chunksize))
    # make space
    if len(cq) >= maxNumChunks:
        cq.pop(0)
    cq.append(nextChunk)
    res = (pd.concat(cq))
    return(res)

# def hasNext(csvStream):
    

INPUT_PATH = "data_processed/maestro/"
add_input_path = lambda x: "{}/{}".format(INPUT_PATH, x)
MUSIC_STREAM_SUBSTR = "maestro_full_music_stream"

music_files = []
for root, dirs, files in os.walk(INPUT_PATH):
    for file in files:
        if MUSIC_STREAM_SUBSTR in file:
            music_files.append(os.path.join(root, file))

### pick first as the music stream
music_files = sorted(music_files) # play in order
print("Number of music streams found:")
print(len(music_files))
print(music_files[:10])

### should only 1 have file to stream
if (len(music_files) > 1):
    whichStream = int(input("Index (0 ... N-1) of stream to pick:"))
else:
    whichStream = 0
musicStream = csvStream(music_files[whichStream])

### Task 0: Stream music audio and display as a webapp

### Task 1: Stream MIDI and display as a webapp

(do audio later after download)

In [ ]:
# import jp_proxy_widget
# from scipy.io import wavfile

""" Demo: auto-updating time series plot, use with voila and watch update """

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

### Setup time series plot
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False, value='plasma',
            options=[{'label': c, 'value': c} for c in px.colors.named_colorscales()])
    ]),
    dcc.Interval(
        id='interval-component',
        interval=1*1000, # update every (this number of) milliseconds
        n_intervals=0
    )
])

### Update above graph
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Input("colorscale-dropdown", "value"),
    Input("interval-component", "n_intervals"))
def update_figure(currBar, colorscale, n_intervals):
    fig = px.scatter(
        # like ggplot with color based on group
        currBar, x="start_sec", y="pitch_mean", color="pitch_mean",
        color_continuous_scale=colorscale, render_mode="webgl", template="plotly_dark",
        title="Mean pitch per sampled bar (streaming)",range_y=[21, 108]).update_traces(mode='lines+markers')
    return(fig)
    
while True:
    currBar = nextChunkWithOverlap(musicStream)
    if currBar is None:
        print(">> End of stream!")
        break

    # Run app and display result inline in the notebook
    app.run_server(mode='inline')

In [ ]:
raise Exception()

### Task 2: Play music and visualize primitive information, construct financial equities, visualize

In [ ]:
""" SPECTROGRAM
    No machine learning required for this.
"""

#

In [ ]:
""" NOTE DENSITY
    No machine learning required for this.
"""

#

### Task 3: Extract music features from MIDI in real-time, construct financial equities, visualize

Sequential learning.
- Validate (try out) against the streamed music audio and series.
- These are the constructed underlyings for financial equities, upon which prediction/regression can work.

Make a local API so other scripts can GET/POST requests (bid/ask) for this.

In [ ]:
""" TEMPO
    Strategy: linear Gaussian state space model / Kalman filter.
    Model tempo (latent variable zt) as a function of notes etc. (observed variables x1 ... xt)
    https://www.researchgate.net/publication/224711190_A_Modified_Kalman_Filtering_Approach_to_On-Line_Musical_Beat_Tracking
"""




In [ ]:
""" HARMONY
"""

#

In [ ]:
""" RHYTHM
"""

#